In [1]:
import pandas as pd
from sqlite_utils import Database

In [2]:
db = Database("mybinder_archive.db")

In [3]:
# get table names
db.table_names()

['mybinderlaunch', 'repo', 'notebook']

In [4]:
launch_table = 'mybinderlaunch'

In [5]:
# db[launch_table].columns
db[launch_table].columns_dict

{'timestamp': str,
 'version': int,
 'origin': str,
 'provider': str,
 'spec': str,
 'org': str,
 'ref': str,
 'resolved_ref': str,
 'r2d_version': str,
 'repo_url': str,
 'repo_id': str}

In [6]:
# NOTE: this notebook wont have same output (nbs_same_output)
query = f'SELECT * FROM {launch_table} ORDER BY RANDOM() LIMIT 100'
pd.read_sql_query(query, db.conn)

,timestamp,version,origin,provider,spec,org,ref,resolved_ref,r2d_version,repo_url,repo_id
0,2019-10-10T11:40:00,3,gke.mybinder.org,GitHub,McKemmish-Group/Workshop3_CHEM3011/master,McKemmish-Group,master,,jupyter/repo2docker:0.10.0-100.g0dba611,https://github.com/mckemmish-group/workshop3_c...,26723
1,2019-02-04T03:32:00,1,mybinder.org,GitHub,ipython/ipython-in-depth/master,ipython,master,,jupyter/repo2docker:ff894f44,https://github.com/ipython/ipython-in-depth,1
2,2020-04-26T22:13:00,3,gke.mybinder.org,Gist,ELC/380e584b87227b15727ec886223d9d4a/master,ELC,master,,jupyter/repo2docker:0.11.0-52.g175b930,https://gist.github.com/elc/380e584b87227b1572...,12808
3,2019-09-18T14:10:00,3,gke.mybinder.org,GitHub,jupyterlab/jupyterlab-demo/try.jupyter.org,jupyterlab,try.jupyter.org,,jupyter/repo2docker:0.10.0-80.g2ee3230,https://github.com/jupyterlab/jupyterlab-demo,4
4,2019-11-05T15:35:00,3,ovh.mybinder.org,GitHub,VincentZ112/conda.git/master,VincentZ112,master,,jupyter/repo2docker:0.10.0-100.g0dba611,https://github.com/vincentz112/conda,29907
...,...,...,...,...,...,...,...,...,...,...,...
95,2019-02-07T07:57:00,1,mybinder.org,GitHub,ipython/ipython-in-depth/master,ipython,master,,jupyter/repo2docker:9c2c7b6a,https://github.com/ipython/ipython-in-depth,1
96,2019-06-19T00:01:00,3,gke.mybinder.org,GitHub,noamross/gams-in-r-course/master,noamross,master,,jupyter/repo2docker:9f110584,https://github.com/noamross/gams-in-r-course,16837
97,2020-02-11T03:05:00,3,turing.mybinder.org,GitHub,ipython/ipython-in-depth/master,ipython,master,,jupyter/repo2docker:0.11.0-5.g8d490cf,https://github.com/ipython/ipython-in-depth,1
98,2019-03-10T18:34:00,1,mybinder.org,GitHub,ipython/ipython-in-depth/master,ipython,master,,jupyter/repo2docker:9766c954,https://github.com/ipython/ipython-in-depth,1


### Total number of launches

In [7]:
db[launch_table].count

9624058

First timestamp

In [8]:
launches = db[launch_table].rows_where(order_by="timestamp")
first_launch_timestamp = next(launches)["timestamp"]
del launches

first_launch_timestamp

'2018-11-03T00:00:00'

Last timestamp

In [9]:
launches = db[launch_table].rows_where(order_by="timestamp desc")
last_launch_timestamp = next(launches)["timestamp"]
del launches

last_launch_timestamp

'2020-06-30T23:59:00'

### Launches per origin

In [10]:
query = f'SELECT origin, count(origin) FROM {launch_table} GROUP BY "origin";'
launches_per_origin = pd.read_sql_query(query, db.conn)
launches_per_origin

,origin,count(origin)
0,binder.mybinder.ovh,556
1,binder.mybinder.turing.ac.uk,129
2,gesis.mybinder.org,681920
3,gke.mybinder.org,4754321
4,gke.mybinder.org:443,1
5,mybinder.org,2741924
6,notebooks.gesis.org,668369
7,ovh.mybinder.org,535332
8,turing.mybinder.org,241506


In [11]:
launches_per_origin_dict = {"gke": 0, "gesis": 0, "turing": 0, "ovh": 0}
for index, row in launches_per_origin.iterrows():
    if row["origin"] == "mybinder.org":
        launches_per_origin_dict["gke"] += row["count(origin)"]
    for origin in launches_per_origin_dict.keys():
        if origin in row["origin"]:
            # print(origin, row["origin"])
            launches_per_origin_dict[origin] += row["count(origin)"]

assert sum(launches_per_origin_dict.values()) == db[launch_table].count

pd.DataFrame.from_dict(
    launches_per_origin_dict, 
    orient="index", 
    columns=["launches"]
).sort_values(by=["launches"], ascending=False)

,launches
gke,7496246
gesis,1350289
ovh,535888
turing,241635


### Launches per provider

In [12]:
query = f'SELECT provider, count(provider) FROM {launch_table} GROUP BY "provider";'
launches_per_provider = pd.read_sql_query(query, db.conn).sort_values(by=["count(provider)"], ascending=False)

launches_per_provider

,provider,count(provider)
4,GitHub,9388681
2,Gist,131204
3,Git,63858
5,GitLab,38556
7,Zenodo,943
6,Hydroshare,383
1,Figshare,306
0,Dataverse,127


In [13]:
# percentage of first 2 providers
(sum(launches_per_provider.head(2)["count(provider)"])*100)/sum(launches_per_provider["count(provider)"])

98.91757718002115

### Repos per provider

In [14]:
query = f'SELECT provider, count(DISTINCT repo_url) FROM {launch_table} GROUP BY "provider";'
repos_per_provider = pd.read_sql_query(query, db.conn).\
                        sort_values(by=["count(DISTINCT repo_url)"], ascending=False)

repos_per_provider

,provider,count(DISTINCT repo_url)
4,GitHub,48774
3,Git,10145
2,Gist,1491
5,GitLab,894
7,Zenodo,86
6,Hydroshare,36
1,Figshare,30
0,Dataverse,14


In [15]:
# # this returns 185 repos for Git provider because they have github url but Git as provider, 
# # which is okay and should work 
# query = f'SELECT provider, count(DISTINCT repo_id) FROM {launch_table} GROUP BY "provider";'
# repos_per_provider = pd.read_sql_query(query, db.conn).\
#                         sort_values(by=["count(DISTINCT repo_id)"], ascending=False)

# repos_per_provider

### Organisations/Users in repos

In [16]:
# number of different orgs
query = "select count(distinct org) from mybinderlaunch;"
db.conn.execute(query).fetchone()[0]

33514